In [10]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


#direct to where the files are downloaded
path =r"C:\Users\heather\Desktop\Post Kaiser Projects\SRTR\data"
os.chdir(path)

In [2]:

files =[r"csrs_final_tables_1905_",r"csrs_final_tables_1911_",
        r"csrs_final_tables_1811_",r"csrs_final_tables_1808_",
        r"csrs_final_tables_1711_"]


oldfiles =[r"csrs_final_tables_1512",r"csrs_final_tables_1606",
          r"csrs_final_tables_1701",r"csrs_final_tables_1707"] 


In [3]:
def Scraper (organlist, filelist, columnslist, sheet, df_name):
    for organ in organlist:
        for file in filelist:
            df = pd.read_excel(file+ organ + ".xls", sheet_name =sheet)
            df.columns = df.columns.str.strip()
            df2 = df[columnslist]
            df2 = df2.iloc[1:]
            df2.CTR_CD = df2.CTR_CD.str.strip()
            df_name = pd.concat([df_name, df2])
    return df_name


In [14]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(path) if isfile(join(path, f))]
#onlyfiles

In [15]:
organcodes = []

for file in onlyfiles:
    organcodes.append(file[-6:-4])
    
organcodes = list(set(organcodes))  
organcodes

['PA', 'LI', 'HL', 'IN', 'LU', 'HR', 'KI', 'KP']

In [16]:
WLvols = pd.DataFrame()


Wcolumns = ['ENTIRE_NAME', 'CTR_CD', 'RELEASE_DATE', 'ORG',
            'WLA_ST_NC1', 'WLA_ADDCEN_NC1', 'WLA_REMDET_NC1' ,'WLA_END_NC1', 'WLA_REMDIED_NC1',
            'WLA_REMOTH_NC1', 'WLA_REMREC_NC1', 'WLA_REMTFER_NC1', 'WLA_REMTXC_NC1', 'WLA_REMTXL_NC1',
            'WLA_REMTXOC_NC1']


WLvols = Scraper(organcodes, files, Wcolumns, "Table B1", WLvols)

In [17]:
Newcolumns = ['Hospital Name', 'Hospital Abbreviation', 'Release Date', 'Organ Code', 
              'On waitlist at start', 'New Listings', 'Deteriorated', 'On waitlist at end',
              'Died', 'Other Reasons', 'Recovered', 'Transferred to another center',
              'Received deceased donor tx', 'Received living donor tx', 'Transplanted at another center'
             ]



WLvols.columns = Newcolumns

In [18]:
WLvols['Release Date'] =pd.to_datetime(WLvols['Release Date'].astype(str).str[:10], infer_datetime_format=True, errors ='coerce')


WLvols['Waitlist Total']= WLvols['On waitlist at start'] + WLvols['New Listings']


In [19]:
JustHeartOneYear =WLvols[(WLvols['Organ Code']=='HR') & (WLvols['Release Date']=='2019-07-07')]
JustHeartOneYear

,Hospital Name,Hospital Abbreviation,Release Date,Organ Code,On waitlist at start,New Listings,Deteriorated,On waitlist at end,Died,Other Reasons,Recovered,Transferred to another center,Received deceased donor tx,Received living donor tx,Transplanted at another center,Waitlist Total
1,Children's of Alabama (ALCH),ALCH,2019-07-07,HR,4,15,0,4,2,0,1,0,12,0,0,19
2,University of Alabama Hospital (ALUA),ALUA,2019-07-07,HR,32,36,4,28,4,5,3,0,24,0,0,68
3,Baptist Medical Center (ARBH),ARBH,2019-07-07,HR,44,18,5,30,3,1,7,0,16,0,0,62
4,Arkansas Children's Hospital (ARCH),ARCH,2019-07-07,HR,0,7,1,1,1,0,1,0,3,0,0,7
5,Phoenix Children's Hospital (AZCH),AZCH,2019-07-07,HR,4,19,2,5,2,0,0,0,14,0,0,23
6,Mayo Clinic Hospital (AZMC),AZMC,2019-07-07,HR,45,62,2,35,1,5,13,0,50,0,1,107
7,Banner University Medical Center-Tucson (AZUA),AZUA,2019-07-07,HR,16,17,3,13,1,3,0,0,13,0,0,33
8,Rady Children's Hospital and Health Center (CACH),CACH,2019-07-07,HR,1,7,1,1,1,0,0,0,5,0,0,8
9,Childrens Hospital Los Angeles (CACL),CACL,2019-07-07,HR,4,13,1,8,1,0,0,0,7,0,0,17
10,Cedars-Sinai Medical Center (CACS),CACS,2019-07-07,HR,115,146,19,114,6,10,7,1,103,0,1,261


In [22]:
level1 = []


for index, row in JustHeartOneYear.iterrows():
    for column in JustHeartOneYear.columns[4:15]:
        if column =='On waitlist at start' or column=='New Listings':
                line = [row['Hospital Abbreviation'],row[column], column]
                level1.append(line)

In [23]:
level2 =[['On waitlist at start', JustHeartOneYear['On waitlist at start'].sum(), 'Waitlist Total'], ['New Listings', JustHeartOneYear['New Listings'].sum(), 'Waitlist Total']]

level2

[['On waitlist at start', 4025, 'Waitlist Total'],
 ['New Listings', 4536, 'Waitlist Total']]

In [24]:
level3 =[]

for column in JustHeartOneYear.columns[6:15]:
    line =['Waitlist Total', JustHeartOneYear[column].sum(), column]
    level3.append(line)

level3

[['Waitlist Total', 351, 'Deteriorated'],
 ['Waitlist Total', 3939, 'On waitlist at end'],
 ['Waitlist Total', 321, 'Died'],
 ['Waitlist Total', 306, 'Other Reasons'],
 ['Waitlist Total', 235, 'Recovered'],
 ['Waitlist Total', 112, 'Transferred to another center'],
 ['Waitlist Total', 3242, 'Received deceased donor tx'],
 ['Waitlist Total', 2, 'Received living donor tx'],
 ['Waitlist Total', 53, 'Transplanted at another center']]

In [25]:
alllevels = level1 + level2 + level3

In [26]:
for level in alllevels:
    print(level[0], str("["), level[1], str(']'), level[2])

ALCH [ 4 ] On waitlist at start
ALCH [ 15 ] New Listings
ALUA [ 32 ] On waitlist at start
ALUA [ 36 ] New Listings
ARBH [ 44 ] On waitlist at start
ARBH [ 18 ] New Listings
ARCH [ 0 ] On waitlist at start
ARCH [ 7 ] New Listings
AZCH [ 4 ] On waitlist at start
AZCH [ 19 ] New Listings
AZMC [ 45 ] On waitlist at start
AZMC [ 62 ] New Listings
AZUA [ 16 ] On waitlist at start
AZUA [ 17 ] New Listings
CACH [ 1 ] On waitlist at start
CACH [ 7 ] New Listings
CACL [ 4 ] On waitlist at start
CACL [ 13 ] New Listings
CACS [ 115 ] On waitlist at start
CACS [ 146 ] New Listings
CALL [ 22 ] On waitlist at start
CALL [ 21 ] New Listings
CAMB [ 0 ] On waitlist at start
CAMB [ 0 ] New Listings
CAPC [ 28 ] On waitlist at start
CAPC [ 30 ] New Listings
CAPM [ 2 ] On waitlist at start
CAPM [ 24 ] New Listings
CASD [ 24 ] On waitlist at start
CASD [ 57 ] New Listings
CASF [ 30 ] On waitlist at start
CASF [ 16 ] New Listings
CASG [ 24 ] On waitlist at start
CASG [ 20 ] New Listings
CASH [ 3 ] On waitlist